<a href="https://colab.research.google.com/github/davivaug2/math4000_machine_learning_2023/blob/main/ml_final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install d2l

In [2]:

import torch
from torch import nn
from torch.nn import functional as F



In [3]:

 # from d2l import torch as d2l

In [4]:


import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
     


cuda:0


In [5]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 150
n_iters = 10000
#batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

#trainset.to(torch.device("cuda:0"))
#trainloader.to(torch.device("cuda:0"))
#testset.to(torch.device("cuda:0"))
#testloader.to(torch.device("cuda:0"))

Files already downloaded and verified
Files already downloaded and verified


In [7]:

import torch.nn as nn
from torch.nn import functional as F

class Residual(nn.Module):
    def __init__(self, in_channel, out_channel, use_1x1Conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, stride=strides)
        # self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn1 = nn.LazyBatchNorm2d()
        self.conv2 = nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, stride=strides)
        # self.bn2 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.LazyBatchNorm2d()

        self.conv3 = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        # self.bn3 = nn.BatchNorm2d(out_channel)
        self.bn3 = nn.LazyBatchNorm2d()

        if use_1x1Conv:
            self.conv4 = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=strides)
        else:
            self.conv4 = None

    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        #out = self.bn2(self.conv2(out))
        out = F.relu(self.bn2(self.conv2(X)))
        out = self.bn3(self.conv3(out))
        if self.conv4:
            X = self.conv4(X)
        out += X
        return F.relu(out)

def residualBlock(in_channel, out_channel, num_residuals, first_block=False):
    blks = []
    for i in range(num_residuals):
        if i==0 and not first_block:
            blks.append(Residual(in_channel, out_channel, use_1x1Conv=True,
            strides=2))
        else:
            blks.append(Residual(out_channel, out_channel))
    
    return blks

class ResNet(nn.Module):
    def __init__(self, input_channel, n_classes):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(input_channel, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        self.b2 = nn.Sequential(*residualBlock(64, 64, 2, first_block=True))
        self.b3 = nn.Sequential(*residualBlock(64, 384, 2))
        self.b4 = nn.Sequential(*residualBlock(384, 768, 2))
        self.b5 = nn.Sequential(*residualBlock(768, 1024, 2))
        self.b6 = nn.Sequential(*residualBlock(1024, 1024, 2))
        self.b7 = nn.Sequential(*residualBlock(1024, 768, 2))
        self.b8 = nn.Sequential(*residualBlock(768, 768, 2))
        self.b9 = nn.Sequential(*residualBlock(768, 768, 2))
        self.b10 = nn.Sequential(*residualBlock(768, 256, 2))

        self.finalLayer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),nn.Linear(256, n_classes))

        self.b1.apply(self.init_weights)
        self.b2.apply(self.init_weights)
        self.b3.apply(self.init_weights)
        self.b4.apply(self.init_weights)
        self.b5.apply(self.init_weights)
        self.b6.apply(self.init_weights)
        self.b7.apply(self.init_weights)
        self.b8.apply(self.init_weights)
        self.b9.apply(self.init_weights)
        self.b10.apply(self.init_weights)
        self.finalLayer.apply(self.init_weights)

    def init_weights(self, layer):
        if type(layer) == nn.Conv2d:
            nn.init.kaiming_normal_(layer.weight, mode='fan_out')
        if type(layer) == nn.Linear:
            nn.init.normal_(layer.weight, std=1e-3)
        if type(layer) == nn.BatchNorm2d:
            nn.init.constant_(layer.weight, 1)
            nn.init.constant_(layer.bias, 0)
        


    def forward(self, X):
        out = self.b1(X)
        out = self.b2(out)
        out = self.b3(out)
        out = self.b4(out)
        out = self.b5(out)

        out = self.b6(out)
        out = self.b7(out)
        out = self.b8(out)
        out = self.b9(out)
        out = self.b10(out)



        out = self.finalLayer(out)

        return out

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
#3 is in_channel, 10 in number of outputs
model = ResNet(3,10)
model.to(torch.device("cuda:0"))
     

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


ResNet(
  (b1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (b2): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): LazyBatchNorm2d(

In [10]:


learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
     


In [11]:
#
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
      # Load images
        images, labels = images.to(device), labels.to(device)
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 250 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 250. Loss: 1.3071235418319702. Accuracy: 51.959999084472656
Iteration: 500. Loss: 1.0641311407089233. Accuracy: 60.29999923706055
Iteration: 750. Loss: 1.035887360572815. Accuracy: 63.44999694824219
Iteration: 1000. Loss: 0.8321187496185303. Accuracy: 67.5
Iteration: 1250. Loss: 0.7875125408172607. Accuracy: 70.02999877929688
Iteration: 1500. Loss: 0.643134593963623. Accuracy: 72.36000061035156
Iteration: 1750. Loss: 0.5252638459205627. Accuracy: 72.83000183105469
Iteration: 2000. Loss: 0.512503981590271. Accuracy: 74.0999984741211
Iteration: 2250. Loss: 0.5846725702285767. Accuracy: 73.58999633789062
Iteration: 2500. Loss: 0.2877025008201599. Accuracy: 75.3699951171875
Iteration: 2750. Loss: 0.38471177220344543. Accuracy: 75.0199966430664
Iteration: 3000. Loss: 0.3463422954082489. Accuracy: 75.43999481201172
Iteration: 3250. Loss: 0.31843501329421997. Accuracy: 74.68000030517578
Iteration: 3500. Loss: 0.10883495211601257. Accuracy: 75.06999969482422
Iteration: 3750. Loss: 0

In [ ]:
# not used

In [6]:

import torch.nn as nn
from torch.nn import functional as F

class Residual(nn.Module):
    def __init__(self, in_channel, out_channel, use_1x1Conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1, stride=strides)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channel)

        if use_1x1Conv:
            self.conv3 = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=strides)
        else:
            self.conv3 = None

    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        if self.conv3:
            X = self.conv3(X)
        out += X
        return F.relu(out)

def residualBlock(in_channel, out_channel, num_residuals, first_block=False):
    blks = []
    for i in range(num_residuals):
        if i==0 and not first_block:
            blks.append(Residual(in_channel, out_channel, use_1x1Conv=True,
            strides=2))
        else:
            blks.append(Residual(out_channel, out_channel))
    
    return blks

class ResNet(nn.Module):
    def __init__(self, input_channel, n_classes):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(input_channel, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        self.b2 = nn.Sequential(*residualBlock(64, 64, 2, first_block=True))
        self.b3 = nn.Sequential(*residualBlock(64, 384, 2))
        self.b4 = nn.Sequential(*residualBlock(384, 768, 2))
        self.b5 = nn.Sequential(*residualBlock(768, 1024, 2))
        self.b6 = nn.Sequential(*residualBlock(1024, 1024, 1))
        self.b7 = nn.Sequential(*residualBlock(1024, 768, 1))
        self.b8 = nn.Sequential(*residualBlock(768, 768, 1))
        self.b9 = nn.Sequential(*residualBlock(768, 768, 1))
        self.b10 = nn.Sequential(*residualBlock(768, 256, 2))

        self.finalLayer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),nn.Linear(256, n_classes))

        self.b1.apply(self.init_weights)
        self.b2.apply(self.init_weights)
        self.b3.apply(self.init_weights)
        self.b4.apply(self.init_weights)
        self.b5.apply(self.init_weights)
        self.b6.apply(self.init_weights)
        self.b7.apply(self.init_weights)
        self.b8.apply(self.init_weights)
        self.b9.apply(self.init_weights)
        self.b10.apply(self.init_weights)
        self.finalLayer.apply(self.init_weights)

    def init_weights(self, layer):
        if type(layer) == nn.Conv2d:
            nn.init.kaiming_normal_(layer.weight, mode='fan_out')
        if type(layer) == nn.Linear:
            nn.init.normal_(layer.weight, std=1e-3)
        if type(layer) == nn.BatchNorm2d:
            nn.init.constant_(layer.weight, 1)
            nn.init.constant_(layer.bias, 0)
        


    def forward(self, X):
        out = self.b1(X)
        out = self.b2(out)
        out = self.b3(out)
        out = self.b4(out)
        out = self.b5(out)

        out = self.b6(out)
        out = self.b7(out)
        out = self.b8(out)
        out = self.b9(out)
        out = self.b10(out)



        out = self.finalLayer(out)

        return out